# Novel AI Image Detection System

## Novel Approaches Implemented:
1. **Physics-Based Lighting Consistency Analysis** - Detects impossible lighting patterns in AI images
2. **Semantic Consistency with CLIP** - Verifies semantic coherence using vision-language models
3. **Neuromorphic Feature Engineering** - Brain-inspired synchrony and complexity measures
4. **Quantum-Inspired Features** - Amplitude and phase-based representation
5. **Multi-Scale Wavelet Analysis** - Deep frequency decomposition
6. **Adversarial Robustness Testing** - Foolbox integration for model hardening
7. **Ensemble with Specialized Classifiers** - CatBoost, XGBoost, MLP, SVM

## Installation Requirements

In [ ]:
!pip install pytorch_wavelets foolbox ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

## Part 1: Feature Extraction Pipeline
This cell contains all feature extraction functions including novel physics-based and semantic features.

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.amp import autocast
import torchvision.models as models
import torchvision.transforms as transforms
from sklearn.decomposition import IncrementalPCA
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
import kornia
from pytorch_wavelets import DWTForward
import gc
import warnings
from joblib import Parallel, delayed
from tqdm import tqdm
from scipy.optimize import least_squares
import numpy.linalg as la

try:
    import clip
    CLIP_AVAILABLE = True
except ImportError:
    print("[WARN] CLIP not available. Install with: pip install git+https://github.com/openai/CLIP.git")
    CLIP_AVAILABLE = False

warnings.filterwarnings("ignore")

# Device configuration
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"[INFO] Device: {DEVICE}")
torch.backends.cudnn.benchmark = True

# Global configuration
USE_DEEP = True
BATCH_SIZE = 576
DEEP_FEATURE_DIM = 128

# Memory management decorator
def memory_cleanup(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()
        return result
    return wrapper

# Deep feature extractor
if USE_DEEP:
    print("[INFO] Loading MobileNetV3 (feature extractor)...")
    mobilenet = models.mobilenet_v3_small(weights='IMAGENET1K_V1').to(DEVICE)
    mobilenet.classifier = nn.Identity()
    mobilenet.eval()
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

@memory_cleanup
def extract_deep_features(img_rgb, pca=None):
    try:
        img_t = transform(img_rgb).unsqueeze(0).to(DEVICE)
        with torch.no_grad(), autocast('cuda' if DEVICE == 'cuda' else 'cpu'):
            feat = mobilenet(img_t)
        feat = feat.cpu().numpy().flatten()
        
        if pca is not None and hasattr(pca, 'components_'):
            feat = pca.transform(feat.reshape(1, -1)).flatten()
            if len(feat) > DEEP_FEATURE_DIM:
                feat = feat[:DEEP_FEATURE_DIM]
            elif len(feat) < DEEP_FEATURE_DIM:
                feat = np.pad(feat, (0, DEEP_FEATURE_DIM - len(feat)))
        else:
            feat = feat[:DEEP_FEATURE_DIM] if len(feat) > DEEP_FEATURE_DIM else np.pad(feat, (0, DEEP_FEATURE_DIM - len(feat)))
        
        return feat
    except Exception as e:
        print(f"[WARN] Deep feature extraction failed: {e}")
        return np.zeros(DEEP_FEATURE_DIM)

# GPU-accelerated feature extractors
@memory_cleanup
def compute_sobel_features(img_tensor):
    try:
        if img_tensor.shape[-2:] < (16, 16):
            return [0.0] * 3, ["sobel_mean", "sobel_std", "sobel_edge_density"]
        gray = kornia.color.rgb_to_grayscale(img_tensor.unsqueeze(0))
        with autocast('cuda' if DEVICE == 'cuda' else 'cpu'):
            sobel = kornia.filters.sobel(gray)
            mag = torch.norm(sobel, dim=1)
        feats = [mag.mean().item(), mag.std().item(), (mag > 0.05).float().mean().item()]
        return feats, ["sobel_mean", "sobel_std", "sobel_edge_density"]
    except Exception as e:
        print(f"[DEBUG] Sobel computation failed: {e}")
        return [0.0] * 3, ["sobel_mean", "sobel_std", "sobel_edge_density"]

@memory_cleanup
def compute_fft_band_energies(img_tensor):
    try:
        if img_tensor.shape[-2:] < (16, 16):
            return [0.0] * 5, ["fft_mean", "fft_std", "fft_low", "fft_mid", "fft_high"]
        
        gray = kornia.color.rgb_to_grayscale(img_tensor.unsqueeze(0))
        gray_2d = gray.squeeze(0).squeeze(0)
        
        with autocast('cuda' if DEVICE == 'cuda' else 'cpu'):
            fft = torch.fft.fft2(gray_2d)
            fft_shift = torch.fft.fftshift(fft)
            mag = torch.log(torch.abs(fft_shift) + 1e-8)
        
        H, W = mag.shape
        if H == 0 or W == 0:
            return [0.0] * 5, ["fft_mean", "fft_std", "fft_low", "fft_mid", "fft_high"]
            
        cy, cx = H // 2, W // 2
        maxr = min(H, W) // 2
        
        if maxr <= 0:
            return [0.0] * 5, ["fft_mean", "fft_std", "fft_low", "fft_mid", "fft_high"]
            
        r1, r2 = max(1, maxr // 4), max(1, maxr // 2)
        
        Y, X = torch.meshgrid(torch.arange(H, device=mag.device), 
                             torch.arange(W, device=mag.device), indexing='ij')
        dist2 = (X - cx) ** 2 + (Y - cy) ** 2
        
        low_mask = dist2 <= r1 ** 2
        mid_mask = (dist2 > r1 ** 2) & (dist2 <= r2 ** 2)
        high_mask = dist2 > r2 ** 2
        
        low = mag[low_mask].mean().item() if low_mask.any() else 0.0
        mid = mag[mid_mask].mean().item() if mid_mask.any() else 0.0
        high = mag[high_mask].mean().item() if high_mask.any() else 0.0
        
        feats = [mag.mean().item(), mag.std().item(), low, mid, high]
        return feats, ["fft_mean", "fft_std", "fft_low", "fft_mid", "fft_high"]
        
    except Exception as e:
        print(f"[DEBUG] FFT computation failed: {e}")
        return [0.0] * 5, ["fft_mean", "fft_std", "fft_low", "fft_mid", "fft_high"]

@memory_cleanup
def compute_lbp_torch(img_tensor, bins=16):
    try:
        if img_tensor.shape[-2:] < (16, 16):
            return [0.0] * bins, [f"lbp_bin{i}" for i in range(bins)]
        
        gray = kornia.color.rgb_to_grayscale(img_tensor.unsqueeze(0))
        gray_2d = gray.squeeze(0).squeeze(0)
        
        pad = F.pad(gray_2d.unsqueeze(0).unsqueeze(0), (1, 1, 1, 1), mode='constant', value=0)
        pad = pad.squeeze(0).squeeze(0)
        
        H, W = gray_2d.shape
        lbp = torch.zeros(H, W, device=gray_2d.device)
        
        offsets = [(-1, -1), (-1, 0), (-1, 1),
                  (0, 1), (1, 1), (1, 0), 
                  (1, -1), (0, -1)]
        
        center = gray_2d[1:H-1, 1:W-1]
        
        for i, (dy, dx) in enumerate(offsets):
            neighbor = pad[1+dy:H-1+dy, 1+dx:W-1+dx]
            lbp[1:H-1, 1:W-1] += ((neighbor >= center) * (2 ** i)).float()
        
        lbp_flat = lbp.flatten().cpu()
        hist = torch.histc(lbp_flat, bins=bins, min=0, max=255)
        hist = hist / (hist.sum() + 1e-8)
        
        return hist.numpy().tolist(), [f"lbp_bin{i}" for i in range(bins)]
        
    except Exception as e:
        print(f"[DEBUG] LBP computation failed: {e}")
        return [0.0] * bins, [f"lbp_bin{i}" for i in range(bins)]

@memory_cleanup
def compute_color_stats(img_tensor):
    """FIXED: Removed emoji character"""
    try:
        if img_tensor.shape[-2:] < (16, 16):
            return [0.0] * 18, [f"{prefix}{i}_{stat}" for prefix in ["rgb", "hsv", "lab"] for i in range(3) for stat in ["mean", "std"]]
        hsv = kornia.color.rgb_to_hsv(img_tensor.unsqueeze(0))
        lab = kornia.color.rgb_to_lab(img_tensor.unsqueeze(0))
        feats, names = [], []
        for space, prefix in zip([img_tensor.unsqueeze(0), hsv, lab], ["rgb", "hsv", "lab"]):
            for i in range(3):
                ch = space[:, i, :, :]  # FIXED: Was ch = space[:, i, :, 🙂
                feats.append(ch.mean().item())
                names.append(f"{prefix}{i}_mean")
                feats.append(ch.std().item())
                names.append(f"{prefix}{i}_std")
        return feats, names
    except Exception as e:
        print(f"[DEBUG] Color stats computation failed: {e}")
        return [0.0] * 18, [f"{prefix}{i}_{stat}" for prefix in ["rgb", "hsv", "lab"] for i in range(3) for stat in ["mean", "std"]]

# Continue with more feature extractors...
print("[INFO] Feature extraction functions loaded successfully!")